In [1]:
from IPython.display import Markdown, Latex
from IPython.display import clear_output
from fastai.vision.all import *
from fastai.vision.widgets import *
import warnings
warnings.filterwarnings('ignore')

# The fake detection shoe system

### Halo dear Sneakerheads, your about to meet the very first app that will check for you if the shoes are Original or Fufu!


From today you do not have to hesitate. Artificial intelligence will decide for you!



1. Take a direct picture of the shoe
2. Click on Upload button
3. Choose the picture you want to evaluate 
4. Wait for the results

#### The system is based on two steps. 

In the first step the model will identify the shoe model (now 5 models are supported, [555088-402, 555088-105, DD1399-101, 555088-109, 861428-061]).

In the second step the system will run a fake detection model for the detected shoe model and display the results

Try it now!

In [2]:
#for windows:
#import pathlib
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

In [3]:
path = Path()

learn_type_shoe = load_learner(path/'model_type_shoe.pkl', cpu=True)
learn_555088_402 = load_learner(path/'model_fake_real_555088-402.pkl', cpu=True)
learn_555088_105 = load_learner(path/'model_fake_real_555088-105.pkl', cpu=True)

btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
out = Output()
threshold = 0.6

In [4]:
def on_data_change(change):
    
    def action_by_shoe_model(shoe_type):
        
        if shoe_type == '555088-105':
            return learn_555088_105.predict(img)
        if shoe_type == '555088-402':
            return learn_555088_402.predict(img)
        if shoe_type == '555088-109':
            return ('Real', 0, [1.000, 0.000])
        if shoe_type == '861428-061':
            return ('Real', 0, [1.000, 0.000])
        if shoe_type == 'DD1399-101':
            return ('Fake', 0, [1.000, 0.000])
        
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    
    with out_pl: 
        
        display(img.to_thumb(128,128))
        pred, pred_idx, probs = learn_type_shoe.predict(img)
        
        if probs[pred_idx]>threshold:
            display(Markdown(f'## Shoe detected. model: {pred}'))
            display(Markdown(f'Prediction: {pred} | Probability: {probs[pred_idx]:.03f}'))
            display(Markdown(f'running fake detection modle for shoe model: {pred}...'))
            pred2, pred_idx2, probs2 = action_by_shoe_model(pred)
            display(Markdown(f'## Prediction : {pred2} Shoe!'))
            display(Markdown(f'Prediction: {pred2} | Probability: {probs2[pred_idx2]:.03f}'))
                
        else:
            out_pl.clear_output()
            display(img.to_thumb(128,128))
            display(Markdown(f'## No shoe detected'))
            display(Markdown('The model can only identify Johnson shoes [555088-402, 555088-105, DD1399-101, 555088-109, 861428-061] models'))

In [5]:
btn_upload.observe(on_data_change, names=['data'])

In [6]:
display(VBox([widgets.Label('Select your Shoe!'), btn_upload, out_pl, lbl_pred]))

##### The project created for a final project upervised by Benny

- Moshiko
- Ziv 